# Тестирование

Тестирование – это проверка, которая позволяет определить: соответствует ли реальное поведение программы ожидаемому, выполнив специально созданный набор тестов.

Тест – это выполнение определенных условий и действий, другая функция или метод, необходимых для проверки работы тестируемой функции/метода/класса или её части.

Литература:
    * Python Unit Testing Framework - https://docs.python.org/2/library/unittest.html
    * Как тестируют в Google - https://www.ozon.ru/context/detail/id/24868052/
    * Документация, специфичная для вашего фреймворка

Вопросы? - _dimkat@gmail.com_

## Пирамида тестирования

![Testing Pyramid](http://www.a1qa.ru/wp-content/uploads/2016/05/test-pyramid.png)

## Unit tests

* test fixture - меры, которые необходимо предпринять перед запуском тестов и после них;
* test case - сам тест, который вызявает проверяемую функцию/метод и проверят соответсвие ожидаемого выхода при известных входах;
* test suite - коллекция тестов или других коллекций тестов;
* test runner - среда управления запуском тестов и доставки их результатов до пользователя.

In [53]:
import unittest

class TestStringMethods(unittest.TestCase):
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())
        
    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

In [54]:
def run_tests(classname):
    class_to_test = classname()
    suite = unittest.TestLoader().loadTestsFromModule(class_to_test)
    unittest.TextTestRunner().run(suite)

In [55]:
run_tests(TestStringMethods)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


Доступные методы: https://docs.python.org/2/library/unittest.html#assert-methods

## setUp() и tearDown()

Тестов может быть много, а подготовка данных к ним занимать много строк кода.

In [56]:
import requests

class RedditParser(object):
    def __init__(self, url="https://www.reddit.com/"):
        self._url = url
        self._text_content = ''
    
    def _GetHTMLBody(self):
        response = requests.get(self._url)
        self._text_content = response.text
        
    def WordOnPage(self, word):
        return True if word in self._text_content else False
        
parser = RedditParser()
parser._GetHTMLBody()
print(parser.WordOnPage("CEO"))

True


In [57]:
_TEST_PAGE_CONTENT = "<html><body><p>I'm a page content.</p></body></html>"

class TestRedditParser(unittest.TestCase):
    def setUp(self):
        self.parser = RedditParser()
        self.parser._text_content = _TEST_PAGE_CONTENT
    
    def testWordOnPage(self):
        self.assertTrue(self.parser.WordOnPage("page"))
    
    def testWordNotOnPage(self):
        self.assertFalse(self.parser.WordOnPage("foo"))
        
run_tests(TestRedditParser)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


Другие методы установки среды выполнения:
    * setUpClass() и tearDownClass();
    * setUpModule() и tearDownModule().

# Mocks

unittest.mock - https://docs.python.org/3/library/unittest.mock.html

Mock - это библиотека, которая позволяет вам заменить часть вашей системы в тестах каким либо ложным, фиктивным объектом и сформулировать утверждение, как он должен был быть использован.

Термины:
    * Mock
    * Fake
    * Stub

```python
from unittest.mock import MagicMock
thing = ProductionClass()
thing.method = MagicMock(return_value=3)
thing.method(3, 4, 5, key='value')
```

```bash
3
```

```python
thing.method.assert_called_with(3, 4, 5, key='value')
```

In [58]:
import requests

class RedditParser(object):
    def __init__(self, url="https://www.reddit.com/"):
        self._url = url
    
    def _GetHTMLBody(self):
        response = requests.get(self._url)
        return response.text
        
    def WordOnPage(self, word):
        return True if word in self._GetHTMLBody() else False
        
parser = RedditParser()
print(parser.WordOnPage("CEO"))

True


In [59]:
import unittest.mock as mock

_TEST_PAGE_CONTENT = "<html><body><p>I'm a page content.</p></body></html>"

class TestRedditParser2(unittest.TestCase):
    def testWordOnPage(self):
        parser = RedditParser()
        with mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT):
            self.assertTrue(parser.WordOnPage("page"))
    
    def testWordNotOnPage(self):
        parser = RedditParser()
        with mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT):
            self.assertFalse(parser.WordOnPage("foo"))

In [60]:
run_tests(TestRedditParser2)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


Изменять поведение объектов - части системы можно разными способами (см. документацию):
    * Прямая перегрузка;
    * setUp/tearDown;
    * Контекстый менеджер;
    * Декоратор.

In [61]:
import unittest.mock as mock

_TEST_PAGE_CONTENT = "<html><body><p>I'm a page content.</p></body></html>"

class TestRedditParser3(unittest.TestCase):
    @mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT)
    def testWordOnPage(self, mock_GetHTMLBody):
        parser = RedditParser()
        with mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT):
            self.assertTrue(parser.WordOnPage("page"))
    
    @mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT)
    def testWordNotOnPage(self, mock_GetHTMLBody):
        parser = RedditParser()
        with mock.patch.object(RedditParser, '_GetHTMLBody', return_value=_TEST_PAGE_CONTENT):
            self.assertFalse(parser.WordOnPage("foo"))

In [62]:
run_tests(TestRedditParser3)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


* Надо изменить поведение целого модуля или библиотеки? -> mock.patch
* Надо изменить поведение функции или метода? -> mock.patch.object